In [34]:
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [35]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [37]:
# read and transform data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# image needs to be resized as per the pre-trained model's parameters
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

train = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
test = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=False)

Files already downloaded and verified


In [38]:
# check image size and labels
for images, labels in train_loader:
  print(images.size(), labels.size())
  break

torch.Size([128, 3, 224, 224]) torch.Size([128])


In [39]:
# pick pre-trained model
model = models.vgg16(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [41]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [42]:
# freeze the network parameters
for param in model.parameters():
  param.requires_grad = False

In [43]:
# remove last fc layer and treat the network as a fixed feature extractor
model.classifier[-1] = nn.Sequential(
    nn.Linear(in_features=4096, out_features=10),
    nn.LogSoftmax(dim = 1)
)

In [44]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=10, bias=True)
    (1): LogSoftmax(dim=1)
  )
)

In [45]:
# the model outputs log of softmax transformation so we can use NLLLoss
criterion = nn.NLLLoss()

In [46]:
for param in model.parameters():
  print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


### Train the Fixed Feature Extractor

In [47]:
from torch.optim import Adam

In [48]:
model.to(device)
optimizer = Adam(model.parameters())

In [49]:
# check labels format
for images,labels in train_loader:
  break

labels

tensor([1, 7, 5, 4, 1, 8, 6, 2, 5, 2, 0, 5, 5, 7, 8, 5, 4, 0, 9, 9, 8, 5, 4, 8,
        5, 6, 7, 0, 6, 9, 1, 8, 8, 3, 3, 4, 6, 0, 2, 7, 3, 5, 2, 4, 1, 4, 0, 8,
        5, 3, 9, 2, 0, 0, 1, 4, 0, 7, 8, 2, 0, 8, 7, 5, 5, 1, 8, 8, 9, 9, 2, 4,
        1, 9, 6, 5, 7, 7, 9, 0, 8, 6, 7, 2, 1, 5, 1, 2, 5, 3, 0, 1, 1, 8, 3, 8,
        2, 5, 0, 6, 0, 4, 8, 1, 5, 6, 5, 6, 4, 7, 3, 6, 5, 3, 9, 8, 0, 9, 6, 0,
        5, 5, 4, 8, 3, 5, 4, 0])

In [50]:
epochs = 1
batch_loss = 0

for epoch in range(epochs):
  for batch_n, (X_train, y_train) in enumerate(train_loader,1):
    X_train = X_train.to(device)
    y_train = y_train.to(device)

    # clear gradients
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    print(f'Epoch({epoch}/{epochs}): Batch({batch_n}/{len(train_loader)}): Loss: {loss.item()}')
  print(f'Training loss : {batch_loss/len(train_loader)}')

Epoch(0/1): Batch(1/391): Loss: 2.4032492637634277
Epoch(0/1): Batch(2/391): Loss: 2.1612887382507324
Epoch(0/1): Batch(3/391): Loss: 1.9288427829742432
Epoch(0/1): Batch(4/391): Loss: 1.7479336261749268
Epoch(0/1): Batch(5/391): Loss: 1.6864545345306396
Epoch(0/1): Batch(6/391): Loss: 1.4751477241516113
Epoch(0/1): Batch(7/391): Loss: 1.440217137336731
Epoch(0/1): Batch(8/391): Loss: 1.298289179801941
Epoch(0/1): Batch(9/391): Loss: 1.1416314840316772
Epoch(0/1): Batch(10/391): Loss: 1.1939246654510498
Epoch(0/1): Batch(11/391): Loss: 1.1375075578689575
Epoch(0/1): Batch(12/391): Loss: 1.011133074760437
Epoch(0/1): Batch(13/391): Loss: 0.9231693744659424
Epoch(0/1): Batch(14/391): Loss: 0.8555132150650024
Epoch(0/1): Batch(15/391): Loss: 0.9213075637817383
Epoch(0/1): Batch(16/391): Loss: 0.7504632472991943
Epoch(0/1): Batch(17/391): Loss: 0.8201310038566589
Epoch(0/1): Batch(18/391): Loss: 0.9495233297348022
Epoch(0/1): Batch(19/391): Loss: 0.6940032243728638
Epoch(0/1): Batch(20/391

In [52]:
CIFAR10_classes = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# move model to CPU
model.to('cpu')
# set to evaluation model to stop tracking gradients
model.eval()

with torch.no_grad():
  X_test, y_test = next(iter(test_loader))
  y_pred = model(X_test)

  # take exp to remove log, this is now just to softmax values
  output = torch.exp(y_pred)

  # take argmax to get index value
  y_pred_class = torch.argmax(output,1)

# reset model to training mode
model.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Evaluate Model

In [54]:
n_corr = (y_test == y_pred_class).sum()
n_total = len(y_pred)

In [57]:
# print accuracy
print(f'Test acc: {n_corr/n_total}')

Test acc: 0.828125
